In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# WebDriver 설정
driver = webdriver.Chrome()
driver.maximize_window()

# 데이터를 저장할 리스트 초기화
data = []

# 페이지 이동 및 정보 수집을 위한 함수
def collect_data():
    branch_list = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".cnt3 .list > ul > li"))
    )
    
    # 각 항목에서 정보 수집
    for branch in branch_list:
        try:
            name = branch.find_element(By.CSS_SELECTOR, ".name").text.strip()
        except:
            name = None
        
        try:
            star = branch.find_element(By.CSS_SELECTOR, ".star span").text.strip()
        except:
            star = None
        
        try:
            company_types = branch.find_elements(By.CSS_SELECTOR, ".companyTypeLabel p")
            domestic = "O" if any(ct.text.strip() == "국" for ct in company_types) else None
            imported = "O" if any(ct.text.strip() == "수" for ct in company_types) else None
            tire = "O" if any(ct.text.strip() == "타" for ct in company_types) else None
        except:
            domestic = imported = tire = None
        
        try:
            address = branch.find_element(By.CSS_SELECTOR, ".text .addr").text.replace("•", "").strip()
        except:
            address = None
        
        try:
            phone = branch.find_element(By.CSS_SELECTOR, ".text a[href^='tel']").text.replace("•", "").strip()
        except:
            phone = None
        
        try:
            response_time = branch.find_element(By.XPATH, ".//p[contains(text(), '평균 답변')]").text.replace("•", "").strip()
        except:
            response_time = None
        
        try:
            review_text = branch.find_element(By.XPATH, ".//p[contains(text(), '후기')]").text
            reviews = review_text.split("후기")[1].split("추천")[0].strip()
            recommendations = review_text.split("추천")[1].strip()
        except:
            reviews = recommendations = None
        
        # 수집한 데이터를 리스트에 저장
        data.append({
            "업체명": name,
            "평점": star,
            "국산차": domestic,
            "수입차": imported,
            "타이어": tire,
            "주소": address,
            "전화번호": phone,
            "평균 답변시간": response_time,
            "후기": reviews,
            "추천": recommendations
        })

# 페이지별로 URL을 직접 변경하여 데이터 수집
for page in range(1, 17):  # 필요한 페이지 범위로 설정
    driver.get(f"https://www.gongim.com/front/html/branchsearch.php?page={page}&g1=1&g2=1&sidoname=&sigunguname=&storename=&order_by=l")
    time.sleep(2) 
    collect_data()

# 데이터프레임 생성
df = pd.DataFrame(data)

# WebDriver 종료
driver.quit()

df

,업체명,평점,국산차,수입차,타이어,주소,전화번호,평균 답변시간,후기,추천
0,광명[하안점],4.6,O,None,None,경기도 광명시 범안로 963,02-897-4040,평균 답변 12시간 이내,177,15
1,서울[독산점],5.0,O,None,None,서울시 금천구 독산로 50,02-805-5109,평균 답변 17시간 이내,62,23
2,서울[시흥점],5.0,O,None,None,서울시 금천구 시흥대로 82 시흥동 1층 102호,02-895-1950,평균 답변 18시간 이내,337,43
3,서울[가산점],5.0,O,None,None,서울시 금천구 가산동 139-10,02-853-8860,평균 답변 5시간 이내,200,36
4,경기[광명점],4.7,O,O,None,경기도 광명시 광명로 948-1(광명동),02-2612-1084,평균 답변 4시간 이내,124,15
...,...,...,...,...,...,...,...,...,...,...
308,부산[장림점],4.8,O,O,None,부산광역시 사하구 장평로22,010-6403-7522,평균 답변 1시간 이내,135,14
309,부산[중동점],5.0,O,O,O,부산광역시 해운대구 좌동로14번길 43,010-2847-6721,평균 답변 9시간 이내,494,37
310,부산[용당점],4.7,O,O,None,부산광역시 남구 용당동 473-18,010-9101-8479,평균 답변 17시간 이내,207,24
311,테스트지점,,O,O,O,경북 울릉군 북면 나리,010-8287-0900,평균 답변 1시간 이내,0,4


In [5]:
df['주소'] = df['주소'].str.replace('특별시|광역시|특별자치도|특별자치시', '', regex=True)
df['주소'] = df['주소'].str.replace('경기도', '경기', regex=False)
df['주소'] = df['주소'].str.replace('강원도', '강원', regex=False)
df['주소'] = df['주소'].str.replace('충청북도', '충북', regex=False)
df['주소'] = df['주소'].str.replace('충청남도', '충남', regex=False)
df['주소'] = df['주소'].str.replace('전라북도', '전북', regex=False)
df['주소'] = df['주소'].str.replace('전라남도', '전남', regex=False)
df['주소'] = df['주소'].str.replace('경상북도', '경북', regex=False)
df['주소'] = df['주소'].str.replace('경상남도', '경남', regex=False)

# '시도' 열에 주소 첫 두 글자 추가
df['시도'] = df['주소'].str[:2]

# '시군구' 열에 첫 번째와 두 번째 띄어쓰기 사이의 텍스트 추출
df['시군구'] = df['주소'].str.extract(r'^\S+\s(\S+)')[0]

df['시도+시군구'] = df['시도'] + df['시군구']

In [7]:
df.to_excel('공임나라_최종.xlsx')